In [2]:
import cv2
import glob
import os

selfy_file = "test_selfy.jpg"

# 返す変数
face_detect_or_not = False
detected_product_path = ""

# 顔画像の学習データ
cascade_path = "haarcascade_frontalface_alt.xml"

selfies_path = os.getcwd() + '/' + "static/selfies/"
products_path = os.getcwd() + '/' + "static/products/"
outputs_path = os.getcwd() + '/' + "static/outputs/"

 #ファイル読み込み
image = cv2.imread(selfies_path + selfy_file)
image_gray = cv2.cvtColor(image, 0)
#カスケード分類器の特徴量を取得する
cascade = cv2.CascadeClassifier(cascade_path)

# 物体認識（顔認識）の実行
# image – CV_8U 型の行列．ここに格納されている画像中から物体が検出されます
# objects – 矩形を要素とするベクトル．それぞれの矩形は，検出した物体を含みます
# scaleFactor – 各画像スケールにおける縮小量を表します
# minNeighbors – 物体候補となる矩形は，最低でもこの数だけの近傍矩形を含む必要があります
# flags – このパラメータは，新しいカスケードでは利用されません．古いカスケードに対しては，
# cvHaarDetectObjects 関数の場合と同じ意味を持ちます
# minSize – 物体が取り得る最小サイズ．これよりも小さい物体は無視されます
facerect = cascade.detectMultiScale(image_gray, scaleFactor=1.1, minNeighbors=1, minSize=(60, 60))

if len(facerect) > 0:
    face_detect_or_not = True

# ****************************************************************************
# 商品画像の検知
img2 = image
gray2 = cv2.cvtColor(img2, 0)
    
# 特徴量記述
detector = cv2.AKAZE_create()

kp2, des2 = detector.detectAndCompute(gray2, None)

# 比較器作成
bf = cv2.BFMatcher(cv2.NORM_HAMMING)

for path in glob.glob(products_path+'*'):
    img1 = cv2.imread(path)
    gray = cv2.cvtColor(img1, 0)
    kp1, des1 = detector.detectAndCompute(gray, None)

# 画像への特徴点の書き込み
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m in matches:
        if m.distance < 70:
            good.append(m)

    if len(good)>10:
        print("マッチしました。%s" % path)
        detected_product_path = path
        break
    else:
        print("マッチしませんでした！")
        #matchesMask = None
        pass


マッチしました。/Users/dulltz/Workspace/felp-server/static/products/calorie_mate.jpg


'/Users/dulltz/Workspace/felp-server/static/products/*'